# Logistic regression with a neural network mindset

## Packages

- [numpy](www.numpy.org) scientific computing with Python.
- [matplotlib](http://matplotlib.org) plot graphs in Python.
- [h5py](http://www.h5py.org) to interact with a dataset that is stored on an H5 file.
- [PIL](http://www.pythonware.com/products/pil/) and [scipy](https://www.scipy.org/) test your model with your own picture.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy 

from PIL import Image
from scipy import ndimage

%matplotlib inline

## Data

Dataset ("data.h5") contains:
- A training set of `m_train` images labeled as cat $(y=1)$ or non-cat. $(y=0)$
- A test set of `m_test` images labeled as cat $(y=1)$ or non-cat. $(y=0)$
- Each image is of shape `(num_px, num_px, 3)` where $3$ is for the $3$ channels. (RGB) 
- Thus, each image is square.

In [ ]:
def load_dataset(train_set_location, test_set_location):
    """
    Load dataset into 4 variables

    Args:
    train_set_location -- location of train set
    test_set_location -- location of test set

    Returns:
    train_set_x_orig --
    train_set_y_orig --
    test_set_x_orig --
    test_set_y_orig --
    classes --
    """

    train_dataset = h5py.File(train_set_location, "r")
    # These data sets are to be pre-preprocessed. (Thus, appended "_orig")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:])  # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:])  # your train set labels

    test_dataset = h5py.File(test_set_location, "r")
    # These data sets are to be pre-preprocessed. (Thus, appended "_orig")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:])  # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:])  # your test set labels

    classes = np.array(test_dataset["list_classes"][:])  # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [ ]:
# Load the data. (cat/non-cat)
train_set_location = 'data/train_catvnoncat.h5'
test_set_location = 'data/test_catvnoncat.h5'
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset(train_set_location, test_set_location)

In [ ]:
# Show an example of a picture.
index = 25
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y[:, index]) + ", it's a '" + classes[np.squeeze(train_set_y[:, index])].decode("utf-8") +  "' picture.")

In [ ]:
# Understand Matrix/Vector dimensions
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y.shape))

In [ ]:
# Reshape the training and test examples such that matrices are flattened into vectors.
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0], -1).T

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

To represent color images, the red, green and blue channels (RGB) must be specified for each pixel, and so the pixel value is actually a vector of three numbers ranging from 0 to 255.

In [ ]:
# center and standardize dataset dividing by the maximum value of a pixel channel
train_set_x = train_set_x_flatten / 255
test_set_x = test_set_x_flatten / 255

<font color='blue'>
    
Common steps for pre-processing a new dataset
- Figure out the dimensions and shapes of the problem. (`m_train`, `m_test`, `num_px`, ...)
- Reshape the datasets such that each example is now a vector of size. `(num_px * num_px * 3, 1)`
- "Standardize" the data.

## Algorithm

<img src="img/LogReg_kiank.png" style="width:650px;height:400px;">

For one example $x^{(i)}$:
$$z^{(i)} = w^T x^{(i)} + b \tag{1}$$
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})\tag{2}$$ 
$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})\tag{3}$$

The cost is computed by summing over all training examples:
$$ J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})\tag{4}$$

<font color='blue'>

The main steps for building a Neural Network
1. Define the model structure. (such as number of input features) 
2. Initialize the model's parameters.
3. Loop:
    - Calculate current loss. (forward propagation)
    - Calculate current gradient. (backward propagation)
    - Update parameters. (gradient descent)

In [ ]:
def sigmoid(z):
    """
    Compute the sigmoid of z.

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """

    s = 1 / (1 + np.exp(-z))
    
    return s

In [ ]:
print ("sigmoid([0, 2]) = " + str(sigmoid(np.array([0,2]))))

In [ ]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- Size of the w vector we want. (or number of parameters in this case)
    
    Returns:
    w -- Initialized vector of shape. (dim, 1)
    b -- Initialized scalar. (corresponds to the bias)
    """
    
    w = np.zeros((dim, 1))
    b = 0
    
    # For image inputs, w will be of shape (num_px * num_px * 3, 1)
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [ ]:
dim = 2
w, b = initialize_with_zeros(dim)
print ("w = " + str(w))
print ("b = " + str(b))

### Forward and Backward propagation

Forward Propagation:
- Get $X$
- Compute $A = \sigma(w^T X + b) = (a^{(1)}, a^{(2)}, ..., a^{(m-1)}, a^{(m)})$
- Calculate the cost function: $J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})$

Backward Propagation: 

$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T\tag{7}$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})\tag{8}$$

In [ ]:
def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- Weights, a numpy array of size. (num_px * num_px * 3, 1)
    b -- Bias, a scalar.
    X -- Data of size. (num_px * num_px * 3, number of examples)
    Y -- True "label" vector (containing 0 if non-cat, 1 if cat) of size. (1, number of examples)

    Return:
    cost -- Negative log-likelihood cost for logistic regression.
    dw -- Gradient of the loss with respect to w, thus same shape as w.
    db -- Gradient of the loss with respect to b, thus same shape as b.
    """
    
    # FORWARD PROPAGATION (FROM X TO COST)
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) + b)  # compute activation
    cost = -(np.dot(Y, np.log(A.T)) + np.dot((1-Y), np.log(1-A.T))) / m  # compute cost
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = np.dot(X, (A - Y).T) / m
    db = np.sum(A - Y) / m

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw, "db": db}
    
    return grads, cost

In [ ]:
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propagate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

### Optimization
- Update the parameters using gradient descent.

In [ ]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm.
    
    Arguments:
    w -- Eeights, a numpy array of size. (num_px * num_px * 3, 1)
    b -- Bias, a scalar.
    X -- Data of shape. (num_px * num_px * 3, number of examples)
    Y -- True "label" vector (containing 0 if non-cat, 1 if cat) of shape. (1, number of examples)
    num_iterations -- Number of iterations of the optimization loop.
    learning_rate -- Learning rate of the gradient descent update rule.
    print_cost -- True to print the loss every 100 steps.
    
    Returns:
    params -- Dictionary containing the weights w and bias b.
    grads -- Dictionary containing the gradients of the weights and bias with respect to the cost function.
    costs -- List of all the costs computed during the optimization, this will be used to plot the learning curve.
    """
    
    costs = []
    
    for i in range(num_iterations):
    
        # Cost and gradient calculation.
        grads, cost = propagate(w, b, X, Y)
        
        # Retrieve derivatives from grads.
        dw = grads["dw"]
        db = grads["db"]
        
        # Update rule.
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # Record the costs.
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations.
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w, "b": b}
    
    grads = {"dw": dw, "db": db}
    
    return params, grads, costs

In [ ]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 100, learning_rate = 0.009, print_cost = False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

### Inference

- Calculate $\hat{Y} = A = \sigma(w^T X + b)$
- Convert the entries of a into 0 (if activation <= 0.5) or 1 (if activation > 0.5), stores the predictions in a vector `Y_prediction`.

In [ ]:
def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters. (w, b)
    
    Arguments:
    w -- Weights, a numpy array of size. (num_px * num_px * 3, 1)
    b -- Bias, a scalar.
    X -- Data of size. (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- A numpy array (vector) containing all predictions (0/1) for the examples in X.
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture.
    A = sigmoid(np.dot(w.T, X) + b) 
    
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i].
        if A[0, i] <= 0.5:
            Y_prediction[0, i] = 0
        else:
            Y_prediction[0, i] = 1
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [ ]:
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(predict(w, b, X)))

### Model

Notation:
- `Y_prediction_test` for the predictions on the test set.
- `Y_prediction_train` for the predictions on the train set.
- `w, costs, grads` for the outputs of `optimize()`.

In [ ]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    """
    Builds the logistic regression model by calling the function you've implemented previously.
    
    Arguments:
    X_train -- Training set represented by a numpy array of shape. (num_px * num_px * 3, m_train)
    Y_train -- Training labels represented by a numpy array of shape. (1, m_train)
    X_test -- Test set represented by a numpy array of shape. (num_px * num_px * 3, m_test)
    Y_test -- Test labels represented by a numpy array of shape. (1, m_test)
    num_iterations -- Hyperparameter representing the number of iterations to optimize the parameters.
    learning_rate -- Hyperparameter representing the learning rate used in the update rule of optimize().
    print_cost -- Set to true to print the cost every 100 iterations.
    
    Returns:
    d -- Dictionary containing information about the model.
    """
    
    # initialize parameters with zeros.
    w, b = initialize_with_zeros(X_train.shape[0])

    # Gradient descent.
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost = False)
    
    # Retrieve parameters w and b from dictionary "parameters".
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples.
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    # Print train/test Errors.
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [ ]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

In [ ]:
# Plot learning curve.
costs = np.squeeze(d['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.title("Learning rate =" + str(d["learning_rate"]))
plt.show()

### Choice of learning rate

- The learning rate $\alpha$  determines how rapidly we update the parameters. 
    - If too large, the cost may oscillate up and down or even diverge.
    - If too small, we will need too many iterations to converge to the best values.

In [ ]:
learning_rates = [0.01, 0.001, 0.0001]
models = {}
for i in learning_rates:
    print ("learning rate is: " + str(i))
    models[str(i)] = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 1500, learning_rate = i, print_cost = False)
    print ('\n' + "-------------------------------------------------------" + '\n')

for i in learning_rates:
    plt.plot(np.squeeze(models[str(i)]["costs"]), label= str(models[str(i)]["learning_rate"]))

plt.ylabel('cost')
plt.xlabel('iterations (hundreds)')

legend = plt.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
plt.show()

- In deep learning, recommendation is 
    - Choose the learning rate that better minimizes the cost function.
    - If the model overfits, use other techniques to reduce overfitting.